In [ ]:
import tensorflow as tf
import numpy as np
from alexnet_model import AlexNet
import matplotlib.pyplot as plt

# Function to prepare data for training, validation, and testing
def train_alexnet():
    # Create data generators
    train_data = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,       # Normalize pixel values to the range [0,1]
        shuffle = True
    )
    
    test_data = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shuffle = True
    )

    valid_data = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale = 1./255
    )
    
    # Loading Training and Validation Data
    train_generator = train_data.flow_from_directory(
        '/Users/maximilianstumpf/Desktop/UCLA/Math156 - Machine Learning/Project/Data/Train1',
        batch_size=128,          # Number of images per batch
        class_mode='categorical',  # One-hot encoding of labels
    )
    
    validation_generator = valid_data.flow_from_directory(
        '/Users/maximilianstumpf/Desktop/UCLA/Math156 - Machine Learning/Project/Data/Val',
        batch_size=128,
        class_mode='categorical',
    )
    
    # Loading Test Data
    test_generator = test_data.flow_from_directory(
        '/Users/maximilianstumpf/Desktop/UCLA/Math156 - Machine Learning/Project/Data/Test',
        batch_size=128,
        class_mode='categorical'
    )
    
    # Create AlexNet Model
    model = AlexNet(num_classes=50)
    
    # Learning Rate scheduler
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate = 0.01,
        decay_steps = 10000,
        decay_rate = 0.96,
        staircase = True
    )

    # Optimizer (SGD)
    optimizer = tf.keras.optimizers.SGD( 
        learning_rate=lr_schedule,
        momentum=0.9,        # Accelerates convergence
        nesterov=True,
    )
    
    # Compiling the model
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',  # Loss function
        metrics=['accuracy']             # Track accuracy
    )
    
    # Training the model
    history = model.fit(
        train_generator,
        epochs=20,
        validation_data=validation_generator,
        callbacks=[
            #tf.keras.callbacks.LearningRateScheduler(
            #    lambda epoch: 0.01 if epoch < 10 else (0.001 if epoch < 15 else 0.0001)  # Dynamic LR adjustment
            #),
            tf.keras.callbacks.ModelCheckpoint(
                'alexnet_checkpoint.keras',  # Save checkpoints
                save_best_only=True,
                monitor='val_accuracy'
            ),
            tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights = True)
        ]
    )
    
    # Save final weights
    model.save_weights('alexnet_final.weights.h5')
    
    # Evaluate on test data
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    return model, history

# Run the training
if __name__ == "__main__":
    model, history = train_alexnet()

Found 152819 images belonging to 50 classes.
Found 38230 images belonging to 50 classes.
Found 47799 images belonging to 50 classes.
Epoch 1/20
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 919s 769ms/step - accuracy: 0.0289 - loss: 4.5800 - val_accuracy: 0.0474 - val_loss: 4.3801
Epoch 2/20
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 842s 705ms/step - accuracy: 0.0497 - loss: 4.3299 - val_accuracy: 0.0600 - val_loss: 4.1882
Epoch 3/20
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 899s 753ms/step - accuracy: 0.0672 - loss: 4.1335 - val_accuracy: 0.0787 - val_loss: 4.0288
Epoch 4/20
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 843s 706ms/step - accuracy: 0.0955 - loss: 3.9183 - val_accuracy: 0.1216 - val_loss: 3.7535
Epoch 5/20
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 855s 716ms/step - accuracy: 0.1475 - loss: 3.6486 - val_accuracy: 0.1781 - val_loss: 3.5648
Epoch 6/20
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 844s 707ms/step - accuracy: 0.2478 - loss: 3.2884 - val_accuracy: 0.2899 - val_loss: 3.2259
Epoch 7/20
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 859s 719ms/step - acc

KeyboardInterrupt: 